In [2]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
import seaborn as sns

from sklearn.mixture import GaussianMixture as GMM
from sklearn.cluster import KMeans

from sklearn.model_selection import StratifiedKFold
from sklearn.cluster import KMeans,AffinityPropagation

In [3]:
def gmm(X,k):
    gmm = GMM(n_components=k, covariance_type='full')
    gmm.fit(X)
    X_predicted = [ [] for i in range(k)]
    for x in X:
        X_predicted[gmm.predict([x])[0]].append(x)
    return X_predicted

In [4]:
def kmeans(X,k):
    kmeans = KMeans(n_clusters=k, init='k-means++')
    kmeans.fit(X)
    X_predicted = [ [] for i in range(k)]
    for x in X:
        X_predicted[kmeans.predict([x])[0]].append(x.tolist())
    return X_predicted

In [5]:
RNASeq_path = 'Replicate2/RNASeq_chrE.txt' 
RPF_path = 'Replicate2/RPF_chrE.txt'

In [6]:
RNASeq = pd.read_csv(RNASeq_path, sep='\t')
RPF = pd.read_csv(RPF_path, sep='\t')

In [7]:
RNASeq.head()

,AccNum,GeneName,cdReads0,cdRPKM0,cdReads1,cdRPKM1,cdReads2,cdRPKM2,cdReads3,cdRPKM3,cdReads4,cdRPKM4
0,NM_017847,ODR4,38.0,4.059277,70.0,6.543270,63.0,5.507023,38.0,2.443294,48.0,2.633165
1,NM_001003803,ATP5S,38.0,8.550792,51.0,10.042101,41.0,7.549494,33.0,4.469549,21.0,2.426687
2,NM_001003800,BICD2,944.0,53.601229,1126.0,55.946484,1067.0,49.576787,711.0,24.299615,752.0,21.927660
3,NM_016649,ESF1,19.0,1.083903,53.0,2.645723,33.0,1.540501,29.0,0.995777,27.0,0.790993
4,NM_016647,THEM6,174.0,40.464994,161.0,32.763309,124.0,23.597344,65.0,9.098517,69.0,8.240452


In [8]:
RPF.head()

,AccNum,GeneName,cdReads0,cdRPKM0,cdReads1,cdRPKM1,cdReads2,cdRPKM2,cdReads3,cdRPKM3,cdReads4,cdRPKM4
0,NM_017847,ODR4,178.0,22.843822,189.0,22.942286,83.0,16.106297,44.0,5.221536,55.0,10.502382
1,NM_001003803,ATP5S,108.0,29.196458,121.0,30.939829,51.0,20.847089,28.0,6.999408,23.0,9.251467
2,NM_001003800,BICD2,516.0,35.199468,576.0,37.165074,300.0,30.944003,274.0,17.283584,278.0,28.216787
3,NM_016649,ESF1,261.0,17.887971,194.0,12.576171,88.0,9.119522,83.0,5.260118,66.0,6.730400
4,NM_016647,THEM6,109.0,30.453722,95.0,25.105195,74.0,31.261833,44.0,11.367459,28.0,11.639874


### Filter for cdReads>=10

In [ ]:
RNASeq = RNASeq[(RNASeq['cdReads0'] >= 10) & (RNASeq['cdReads1'] >= 10) 
                & (RNASeq['cdReads2'] >= 10)& (RNASeq['cdReads3'] >= 10)
                & (RNASeq['cdReads4'] >= 10)]
RPF = RPF[(RPF['cdReads0'] >= 10) 
          & (RPF['cdReads1'] >= 10) 
          & (RPF['cdReads2'] >= 10)
          & (RPF['cdReads3'] >= 10)& (RPF['cdReads4'] >= 10)]

### Apply log2

In [10]:
RNASeq[['cdRPKM0', 'cdRPKM1', 'cdRPKM2', 'cdRPKM3', 'cdRPKM4']] = RNASeq[['cdRPKM0', 
                                                                          'cdRPKM1', 
                                                                          'cdRPKM2', 
                                                                          'cdRPKM3', 'cdRPKM4']].apply(np.log2)

In [11]:
RPF[['cdRPKM0', 'cdRPKM1', 'cdRPKM2', 'cdRPKM3', 'cdRPKM4']] = RPF[['cdRPKM0', 
                                                                    'cdRPKM1', 
                                                                    'cdRPKM2', 
                                                                    'cdRPKM3', 'cdRPKM4']].apply(np.log2)

In [12]:
RNASeq['log2cdRPKM1']=RNASeq['cdRPKM1']-RNASeq['cdRPKM0']
RNASeq['log2cdRPKM2']=RNASeq['cdRPKM2']-RNASeq['cdRPKM0']
RNASeq['log2cdRPKM3']=RNASeq['cdRPKM3']-RNASeq['cdRPKM0']
RNASeq['log2cdRPKM4']=RNASeq['cdRPKM4']-RNASeq['cdRPKM0']

In [13]:
RPF['log2cdRPKM1']=RPF['cdRPKM1']-RPF['cdRPKM0']
RPF['log2cdRPKM2']=RPF['cdRPKM2']-RPF['cdRPKM0']
RPF['log2cdRPKM3']=RPF['cdRPKM3']-RPF['cdRPKM0']
RPF['log2cdRPKM4']=RPF['cdRPKM4']-RPF['cdRPKM0']

In [14]:
RNASeq.head()

,AccNum,GeneName,cdReads0,cdRPKM0,cdReads1,cdRPKM1,cdReads2,cdRPKM2,cdReads3,cdRPKM3,cdReads4,cdRPKM4,log2cdRPKM1,log2cdRPKM2,log2cdRPKM3,log2cdRPKM4
0,NM_017847,ODR4,38.0,2.021223,70.0,2.710012,63.0,2.461273,38.0,1.288828,48.0,1.396798,0.688789,0.440050,-0.732395,-0.624425
1,NM_001003803,ATP5S,38.0,3.096058,51.0,3.327989,41.0,2.916380,33.0,2.160129,21.0,1.278988,0.231931,-0.179678,-0.935929,-1.817070
2,NM_001003800,BICD2,944.0,5.744194,1126.0,5.805976,1067.0,5.631593,711.0,4.602862,752.0,4.454680,0.061781,-0.112601,-1.141333,-1.289514
3,NM_016649,ESF1,19.0,0.116236,53.0,1.403662,33.0,0.623400,29.0,-0.006106,27.0,-0.338264,1.287426,0.507164,-0.122342,-0.454500
4,NM_016647,THEM6,174.0,5.338602,161.0,5.034009,124.0,4.560553,65.0,3.185631,69.0,3.042723,-0.304593,-0.778050,-2.152971,-2.295879


In [15]:
RPF.head()

,AccNum,GeneName,cdReads0,cdRPKM0,cdReads1,cdRPKM1,cdReads2,cdRPKM2,cdReads3,cdRPKM3,cdReads4,cdRPKM4,log2cdRPKM1,log2cdRPKM2,log2cdRPKM3,log2cdRPKM4
0,NM_017847,ODR4,178.0,4.513732,189.0,4.519937,83.0,4.009553,44.0,2.384474,55.0,3.392645,0.006205,-0.504179,-2.129258,-1.121087
1,NM_001003803,ATP5S,108.0,4.867721,121.0,4.951393,51.0,4.381774,28.0,2.807233,23.0,3.209682,0.083672,-0.485947,-2.060489,-1.658039
2,NM_001003800,BICD2,516.0,5.137482,576.0,5.215876,300.0,4.951588,274.0,4.111330,278.0,4.818482,0.078394,-0.185894,-1.026151,-0.319000
3,NM_016649,ESF1,261.0,4.160918,194.0,3.652621,88.0,3.188958,83.0,2.395095,66.0,2.750692,-0.508297,-0.971960,-1.765823,-1.410226
4,NM_016647,THEM6,109.0,4.928547,95.0,4.649914,74.0,4.966330,44.0,3.506838,28.0,3.541004,-0.278633,0.037784,-1.421709,-1.387543


# TE

### ONLY RUN FOR MEDIAN NORMALIZATION

In [18]:
RNASeq['log2cdRPKM1']-=np.median(RNASeq['log2cdRPKM1'])
RNASeq['log2cdRPKM2']-=np.median(RNASeq['log2cdRPKM2'])
RNASeq['log2cdRPKM3']-=np.median(RNASeq['log2cdRPKM3'])
RNASeq['log2cdRPKM4']-=np.median(RNASeq['log2cdRPKM4'])

In [19]:
filteredRNASeq = RNASeq[(RNASeq['log2cdRPKM1'] <= 0.5) 
                        & (RNASeq['log2cdRPKM2'] <= 0.5)
                        & (RNASeq['log2cdRPKM3'] <= 0.5)
                        & (RNASeq['log2cdRPKM4'] <= 0.5)]
filteredRNASeq = filteredRNASeq[(filteredRNASeq['log2cdRPKM1'] >= -0.5) 
                                & (filteredRNASeq['log2cdRPKM2'] >= -0.5)
                                & (filteredRNASeq['log2cdRPKM3'] >= -0.5)
                                & (filteredRNASeq['log2cdRPKM4'] >= -0.5)]

In [20]:
filteredRNASeq.shape

(6199, 16)

In [21]:
filteredRNASeq.head()

,AccNum,GeneName,cdReads0,cdRPKM0,cdReads1,cdRPKM1,cdReads2,cdRPKM2,cdReads3,cdRPKM3,cdReads4,cdRPKM4,log2cdRPKM1,log2cdRPKM2,log2cdRPKM3,log2cdRPKM4
1,NM_001003803,ATP5S,38.0,3.096058,51.0,3.327989,41.0,2.916380,33.0,2.160129,21.0,1.278988,0.267619,0.146942,0.438254,-0.042023
2,NM_001003800,BICD2,944.0,5.744194,1126.0,5.805976,1067.0,5.631593,711.0,4.602862,752.0,4.454680,0.097469,0.214019,0.232850,0.485533
7,NM_016640,MRPS30,315.0,5.120866,312.0,4.914494,296.0,4.741809,216.0,3.844151,202.0,3.518410,-0.170685,-0.052437,0.097466,0.172590
9,NM_001304467,EEF1AKMT2,60.0,3.768442,53.0,3.396905,46.0,3.095810,38.0,2.377083,27.0,1.654979,-0.335849,-0.346011,-0.017176,-0.338416
10,NM_001080825,TMEM120B,36.0,2.363552,42.0,2.393378,46.0,2.427886,23.0,0.984794,20.0,0.554095,0.065514,0.390954,-0.004576,-0.034410


In [22]:
TE = reduce(lambda left,right: pd.merge(left,right,on=['AccNum','GeneName']), [RPF,filteredRNASeq])

In [23]:
TE.shape

(6175, 30)

In [24]:
TE.head()

,AccNum,GeneName,cdReads0_x,cdRPKM0_x,cdReads1_x,cdRPKM1_x,cdReads2_x,cdRPKM2_x,cdReads3_x,cdRPKM3_x,...,cdReads2_y,cdRPKM2_y,cdReads3_y,cdRPKM3_y,cdReads4_y,cdRPKM4_y,log2cdRPKM1_y,log2cdRPKM2_y,log2cdRPKM3_y,log2cdRPKM4_y
0,NM_001003803,ATP5S,108.0,4.867721,121.0,4.951393,51.0,4.381774,28.0,2.807233,...,41.0,2.916380,33.0,2.160129,21.0,1.278988,0.267619,0.146942,0.438254,-0.042023
1,NM_001003800,BICD2,516.0,5.137482,576.0,5.215876,300.0,4.951588,274.0,4.111330,...,1067.0,5.631593,711.0,4.602862,752.0,4.454680,0.097469,0.214019,0.232850,0.485533
2,NM_016640,MRPS30,413.0,5.776360,368.0,5.529620,158.0,4.986657,131.0,4.006829,...,296.0,4.741809,216.0,3.844151,202.0,3.518410,-0.170685,-0.052437,0.097466,0.172590
3,NM_001304467,EEF1AKMT2,55.0,3.907614,58.0,3.903932,23.0,3.246331,15.0,1.920189,...,46.0,3.095810,38.0,2.377083,27.0,1.654979,-0.335849,-0.346011,-0.017176,-0.338416
4,NM_001080825,TMEM120B,56.0,3.265685,75.0,3.606845,23.0,2.578407,23.0,1.868936,...,46.0,2.427886,23.0,0.984794,20.0,0.554095,0.065514,0.390954,-0.004576,-0.034410


In [25]:
TE['cdRPKM0_x'] = TE['cdRPKM0_x']-TE['cdRPKM0_y']
TE['cdRPKM1_x'] = TE['cdRPKM1_x']-TE['cdRPKM1_y']
TE['cdRPKM2_x'] = TE['cdRPKM2_x']-TE['cdRPKM2_y']
TE['cdRPKM3_x'] = TE['cdRPKM3_x']-TE['cdRPKM3_y']
TE['cdRPKM4_x'] = TE['cdRPKM4_x']-TE['cdRPKM4_y']
for i in range(0,5):
    TE.rename(columns={'cdRPKM'+str(i)+'_x':'TE'+str(i)}, inplace=True)
TE = TE[['AccNum', 'GeneName','TE0','TE1','TE2','TE3','TE4']]

In [26]:
TE['foldTE1']=TE['TE1']-TE['TE0']
TE['foldTE2']=TE['TE2']-TE['TE0']
TE['foldTE3']=TE['TE3']-TE['TE0']
TE['foldTE4']=TE['TE4']-TE['TE0']

In [27]:
TE.head()

,AccNum,GeneName,TE0,TE1,TE2,TE3,TE4,foldTE1,foldTE2,foldTE3,foldTE4
0,NM_001003803,ATP5S,1.771663,1.623404,1.465394,0.647103,1.930694,-0.148259,-0.306269,-1.124560,0.159031
1,NM_001003800,BICD2,-0.606712,-0.590100,-0.680005,-0.491531,0.363802,0.016612,-0.073293,0.115181,0.970514
2,NM_016640,MRPS30,0.655493,0.615126,0.244848,0.162678,0.974128,-0.040367,-0.410645,-0.492815,0.318635
3,NM_001304467,EEF1AKMT2,0.139172,0.507027,0.150521,-0.456894,1.292490,0.367854,0.011348,-0.596067,1.153317
4,NM_001080825,TMEM120B,0.902133,1.213467,0.150521,0.884143,2.177961,0.311334,-0.751612,-0.017991,1.275828


In [28]:
TE.keys()

Index(['AccNum', 'GeneName', 'TE0', 'TE1', 'TE2', 'TE3', 'TE4', 'foldTE1',
       'foldTE2', 'foldTE3', 'foldTE4'],
      dtype='object')

In [29]:
TE.shape

(6175, 11)

In [30]:
TE.to_csv('Replicate2/TE_chrE_filtered.txt',sep='\t',index=False)

In [31]:
TE.head()

,AccNum,GeneName,TE0,TE1,TE2,TE3,TE4,foldTE1,foldTE2,foldTE3,foldTE4
0,NM_001003803,ATP5S,1.771663,1.623404,1.465394,0.647103,1.930694,-0.148259,-0.306269,-1.124560,0.159031
1,NM_001003800,BICD2,-0.606712,-0.590100,-0.680005,-0.491531,0.363802,0.016612,-0.073293,0.115181,0.970514
2,NM_016640,MRPS30,0.655493,0.615126,0.244848,0.162678,0.974128,-0.040367,-0.410645,-0.492815,0.318635
3,NM_001304467,EEF1AKMT2,0.139172,0.507027,0.150521,-0.456894,1.292490,0.367854,0.011348,-0.596067,1.153317
4,NM_001080825,TMEM120B,0.902133,1.213467,0.150521,0.884143,2.177961,0.311334,-0.751612,-0.017991,1.275828


In [32]:
X = TE[['foldTE1',
       'foldTE2', 'foldTE3', 'foldTE4']].values

In [33]:
X[:5]

array([[-0.14825931, -0.30626918, -1.12455994,  0.1590308 ],
       [ 0.01661247, -0.0732925 ,  0.11518137,  0.97051427],
       [-0.04036743, -0.4106451 , -0.49281521,  0.31863483],
       [ 0.36785421,  0.01134836, -0.59606679,  1.15331714],
       [ 0.31133413, -0.75161245, -0.01799068,  1.27582795]])

In [34]:
# def plot_heatmap(X,idx):
#     plt.subplot(3, 2, idx+1)
# #     plt.figure()
#     sns.heatmap(X,xticklabels=['foldTE1',
#        'foldTE2', 'foldTE3', 'foldTE4'])
#     plt.title('Heatmap'+str(idx+1))

# list_nums = [5]   
# for num in list_nums:
#     clusters = kmeans(X,num)
#     idx=0
#     for cluster in clusters:
#         print(len(cluster))
#         plot_heatmap(cluster)
#         idx+=1
        
list_nums = [4,5,6]
for num in list_nums:
    clusters = kmeans(X,num) 
    for idx,cluster in enumerate(clusters):
        print(len(cluster))
        genes=[]
        acc_nums=[]
        for row in cluster:
            temp = TE.loc[(TE['foldTE1'] == row[0])& (TE['foldTE2']== row[1])
                          & (TE['foldTE3']== row[2])]
            if(not temp.empty):
                genes.append(temp['GeneName'].values[0])
                acc_nums.append(temp['AccNum'].values[0])
        print(len(cluster),len(genes))
        with open('/Users/akankshitadash/Desktop/Replicate2/'+str(num)+
                  '/Gene'+str(idx+1)+'.txt','w') as f:
            for gene in genes:
                f.write("%s\n" % gene)
        with open('/Users/akankshitadash/Desktop/Replicate2/'+str(num)+
                  '/AccNum'+str(idx+1)+'.txt','w') as f:
            for acc_num in acc_nums:
                f.write("%s\n" % acc_num)


1686
1686 1686
1393
1393 1393
447
447 447
2649
2649 2649
2105
2105 2105
253
253 253
1030
1030 1030
1831
1831 1831
956
956 956
1199
1199 1199
960
960 960
240
240 240
873
873 873
2018
2018 2018
885
885 885


In [ ]:
# def plot_heatmap(X):
#     plt.figure()
#     sns.heatmap(X,xticklabels=['TE0', 'TE1', 'TE2', 'TE3', 'TE4', 'foldTE1',
#        'foldTE2', 'foldTE3', 'foldTE4'])
#     plt.title('Heatmap')
#     plt.savefig('NonChrE/KMeansOutputTE/Heatmap'+str(len(X)))
#     plt.show()

# list_nums = [5]   
# for num in list_nums:
#     clusters = kmeans(X,num)
#     idx=0
#     for cluster in clusters:
#         print(len(cluster))
#         plot_heatmap(cluster)
#         idx+=1

# RPF

In [ ]:
RPF.keys()

In [ ]:
RPF.head()

In [ ]:
RPF_new = reduce(lambda left,right: pd.merge(left,right,on=['AccNum','GeneName']), [RPF,filteredRNASeq])

In [ ]:
RPF_new.head()
for i in range(0,5):
    RPF_new.rename(columns={'cdRPKM'+str(i)+'_x':'cdRPKM'+str(i)}, inplace=True)
RPF_new = RPF_new[['AccNum', 'GeneName','cdRPKM0','cdRPKM1','cdRPKM2','cdRPKM3','cdRPKM4']]

In [ ]:
RPF_new.to_csv('/Users/akankshitadash/Desktop/RPF_NonchrE.txt',sep='\t',index=False)

In [ ]:
RPF_new.shape

In [ ]:
X = RPF_new[['cdRPKM0', 'cdRPKM1', 'cdRPKM2', 'cdRPKM3', 'cdRPKM4']].values

In [ ]:
list_nums = [4,5,6]
for num in list_nums:
    clusters = kmeans(X,num)
    for cluster in clusters:
        print(len(cluster))
    for idx,cluster in enumerate(clusters):
        genes=[]
        acc_nums=[]
        for row_nd in cluster:
            row = row_nd[0].tolist()
            temp = RPF.loc[(RPF['cdRPKM0'] == row[0])& (RPF['cdRPKM1']== row[1]) & (RPF['cdRPKM2']== row[2])]
            if(not temp.empty):
                genes.append(temp['GeneName'].values[0])
                acc_nums.append(temp['AccNum'].values[0])
        print(len(cluster),len(genes))
        with open('NonChrE/KMeansOutputRPF/'+str(num)+'/Gene'+str(idx+1)+'.txt','w') as f:
            for gene in genes:
                f.write("%s\n" % gene)
        with open('NonChrE/KMeansOutputRPF/'+str(num)+'/AccNum'+str(idx+1)+'.txt','w') as f:
            for acc_num in acc_nums:
                f.write("%s\n" % acc_num)

In [ ]:
import random
def plot_heatmap(X):
    plt.figure()
    sns.heatmap(X,xticklabels=['cdRPKM0', 'cdRPKM1', 'cdRPKM2', 'cdRPKM3', 'cdRPKM4'])
    plt.title('Heatmap')
    plt.savefig('NonChrE/KMeansOutputRPF/Heatmap'+str(len(X)))
    plt.show()

list_nums = [5]   
for num in list_nums:
    clusters = kmeans(X,num)
    idx=0
    for cluster in clusters:
        print(len(cluster))
        plot_heatmap(cluster)
        idx+=1

In [ ]:
RNASeq['log2cdRPKM1']

# Median Normalize

In [ ]:
plt.hist(RPF['log2cdRPKM1'], bins = np.arange(-2.0,2.0,0.1))
print(np.median(RPF['log2cdRPKM1']))
plt.xlabel('log2fold values RPF')
plt.ylabel('Frequency')
plt.title('log2fold: 2 hpi/0hpi')
# plt.axis(-2,2)
plt.grid(True)

plt.show()

In [ ]:
plt.hist(RPF['log2cdRPKM2'], bins = np.arange(-2.0,2.0,0.1))
print(np.median(RPF['log2cdRPKM2']))

plt.xlabel('log2fold values RPF')
plt.ylabel('Frequency')
plt.title('log2fold: 4 hpi/0hpi')
# plt.axis(-2,2)
plt.grid(True)

plt.show()

In [ ]:

plt.hist(RPF['log2cdRPKM3'], bins = np.arange(-2.0,2.0,0.1))
print(np.median(RPF['log2cdRPKM3']))


plt.xlabel('log2fold values RPF')
plt.ylabel('Frequency')
plt.title('log2fold: 6 hpi/0hpi')
# plt.axis(-2,2)
plt.grid(True)

plt.show()

In [ ]:

plt.hist(RPF['log2cdRPKM4'], bins = np.arange(-2.0,2.0,0.1))
print(np.median(RPF['log2cdRPKM4']))

plt.xlabel('log2fold values RPF')
plt.ylabel('Frequency')
plt.title('log2fold: 8 hpi/0hpi')
# plt.axis(-2,2)
plt.grid(True)

plt.show()

In [ ]:
RNASeq_2['log2cdRPKM1']=RNASeq_2['cdRPKM1']-RNASeq_2['cdRPKM0']
RNASeq_2['log2cdRPKM2']=RNASeq_2['cdRPKM2']-RNASeq_2['cdRPKM0']
RNASeq_2['log2cdRPKM3']=RNASeq_2['cdRPKM3']-RNASeq_2['cdRPKM0']
RNASeq_2['log2cdRPKM4']=RNASeq_2['cdRPKM4']-RNASeq_2['cdRPKM0']

In [ ]:
RNASeq.head(5)

In [ ]:
RNASeq_2.head(5)

In [ ]:
filteredRNASeq = RNASeq_2[(RNASeq_2['log2cdRPKM1'] <= 0.5) 
                        & (RNASeq_2['log2cdRPKM2'] <= 0.5)
                        & (RNASeq_2['log2cdRPKM3'] <= 0.5)
                        & (RNASeq_2['log2cdRPKM4'] <= 0.5)]
filteredRNASeq = filteredRNASeq[(filteredRNASeq['log2cdRPKM1'] >= -0.5) 
                                & (filteredRNASeq['log2cdRPKM2'] >= -0.5)
                                & (filteredRNASeq['log2cdRPKM3'] >= -0.5)
                                & (filteredRNASeq['log2cdRPKM4'] >= -0.5)]



In [ ]:
filteredRNASeq = RNASeq[(RNASeq['log2cdRPKM1'] <= 0.5) 
                        & (RNASeq['log2cdRPKM2'] <= 0.5)
                        & (RNASeq['log2cdRPKM3'] <= 0.5)
                        & (RNASeq['log2cdRPKM4'] <= 0.5)]
filteredRNASeq = filteredRNASeq[(filteredRNASeq['log2cdRPKM1'] >= -0.5) 
                                & (filteredRNASeq['log2cdRPKM2'] >= -0.5)
                                & (filteredRNASeq['log2cdRPKM3'] >= -0.5)
                                & (filteredRNASeq['log2cdRPKM4'] >= -0.5)]



In [ ]:
filteredRNASeq.head(5)

In [ ]:
print(filteredRNASeq.shape)

In [ ]:
TE = reduce(lambda left,right: pd.merge(left,right,on=['AccNum','GeneName']), [RPF,RNASeq])
TE.head()

In [ ]:
TE.shape

In [ ]:
TE['cdRPKM0_x'] = TE['cdRPKM0_x']-TE['cdRPKM0_y']
TE['cdRPKM1_x'] = TE['cdRPKM1_x']-TE['cdRPKM1_y']
TE['cdRPKM2_x'] = TE['cdRPKM2_x']-TE['cdRPKM2_y']
TE['cdRPKM3_x'] = TE['cdRPKM3_x']-TE['cdRPKM3_y']
TE['cdRPKM4_x'] = TE['cdRPKM4_x']-TE['cdRPKM4_y']
for i in range(0,5):
    TE.rename(columns={'cdRPKM'+str(i)+'_x':'TE'+str(i)}, inplace=True)
TE = TE[['AccNum', 'GeneName','TE0','TE1','TE2','TE3','TE4']]
TE.head()

In [ ]:
TE['foldTE1']=TE['TE1']-TE['TE0']
TE['foldTE2']=TE['TE2']-TE['TE0']
TE['foldTE3']=TE['TE3']-TE['TE0']
TE['foldTE4']=TE['TE4']-TE['TE0']

In [ ]:
TE.head(5)

In [ ]:
plt.hist(TE['foldTE1'], bins = np.arange(-2.0,2.0,0.1))


plt.xlabel('log2fold values TE')
plt.ylabel('Frequency')
plt.title('log2fold: 2 hpi/0hpi')
# plt.axis(-2,2)
plt.grid(True)

plt.show()

In [ ]:
plt.hist(TE['foldTE2'], bins = np.arange(-2.0,2.0,0.1))


plt.xlabel('log2fold values TE')
plt.ylabel('Frequency')
plt.title('log2fold: 4 hpi/0hpi')
# plt.axis(-2,2)
plt.grid(True)

plt.show()

In [ ]:
plt.hist(TE['foldTE3'], bins = np.arange(-2.0,2.0,0.1))


plt.xlabel('log2fold values TE')
plt.ylabel('Frequency')
plt.title('log2fold: 6 hpi/0hpi')
# plt.axis(-2,2)
plt.grid(True)

plt.show()

In [ ]:
plt.hist(TE['foldTE4'], bins = np.arange(-2.0,2.0,0.1))


plt.xlabel('log2fold values TE')
plt.ylabel('Frequency')
plt.title('log2fold: 8 hpi/0hpi')
# plt.axis(-2,2)
plt.grid(True)

plt.show()

In [ ]:
TE.to_csv('/Users/akankshitadash/Desktop/TE_medianNormalize.txt',sep='\t',index=False)

In [ ]:
previous = 'AdjustedRPKMOutput/RPF_chrE/geneTXCD_RPKMoutput_EVadjusted_ReadOutput_'
timepoints = ['ATCACG-s_7_1_genome.txt','TTAGGC-s_7_1_genome.txt',
             'CAGATC-s_7_1_genome.txt','GCCAAT-s_7_1_genome.txt',
             'CTTGTA-s_7_1_genome.txt'
            ]
dataframes = []
for idx,file in enumerate(timepoints):
    df = pd.read_csv(previous+file, sep="	", na_values=['-'])
    df = df.dropna(how='any')
    df = df[['AccNum', 'GeneName','cdReads',
       'cdRPKM']]
    df.rename(columns={'cdRPKM':'cdRPKM'+str(idx), 'cdReads':'cdReads'+str(idx)}, inplace=True)
    dataframes.append(df)

In [ ]:
previous = 'AdjustedRPKMOutput/RNASeq_chrE/geneTXCD_RPKMoutput_EVadjusted_ReadOutput_'
timepoints = ['ATCACG-s_6_1_genome.txt','TTAGGC-s_6_1_genome.txt',
             'CAGATC-s_6_1_genome.txt','GCCAAT-s_6_1_genome.txt',
             'CTTGTA-s_6_1_genome.txt'
            ]
dataframes = []
for idx,file in enumerate(timepoints):
    df = pd.read_csv(previous+file, sep="	", na_values=['-'])
    df = df.dropna(how='any')
    df = df[['AccNum', 'GeneName','cdReads',
       'cdRPKM']]
    df.rename(columns={'cdRPKM':'cdRPKM'+str(idx), 'cdReads':'cdReads'+str(idx)}, inplace=True)
    dataframes.append(df)

In [ ]:
df_rnaseq = reduce(lambda left,right: pd.merge(left,right,on=['AccNum','GeneName']), dataframes)
df_rnaseq.to_csv('AdjustedRPKMOutput/RNASeq_chrE.txt',sep='\t',index=False)